## Setup

In [1]:
import pandas as pd
import numpy as np
import os
import sys
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random
import logging

# Change to project directory
os.chdir("..")

from llmgrapher.helpers.df_helpers import df2Graph, graph2Df, documents2Dataframe

2024-05-12 23:25:02.186 | INFO     | llmgrapher.config:<module>:11 - PROJ_ROOT path is: /home/kleptotrace/Projects/git/llmgrapher
/home/kleptotrace/miniforge3/envs/knowledge-graph/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Logger setup
logger = logging.getLogger("__name__")
logger.setLevel(logging.DEBUG) # only log the current logger in DEBUG mode
logging.basicConfig(
    level=logging.CRITICAL,  # show only CRITICAL messages from other loggers
    format="%(asctime)s [%(levelname)s] %(message)s",
    encoding="utf-8",
    handlers=[   # output log messages to both file and stdout
        logging.FileHandler("logs/extract_graph_notebook.log"),
        logging.StreamHandler(sys.stdout)
    ]
)

In [3]:
# Directory Setup
# Input data directory
data_dir = "ag_news"
inputdirectory = Path(f"./data/raw/input/{data_dir}")
# This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data/interim/output/{out_dir}")

In [4]:
ag_news = pd.read_csv("data/raw/ag_news_train.csv", header=None)

In [5]:
# ag_news_txt = ag_news[2].str.cat(sep="\n")
ag_news_txt = ag_news[2].iloc[:50].str.cat(sep="\n")

In [6]:
with open("data/raw/input/ag_news/ag_news.txt", "w") as f:
    f.write(ag_news_txt)

## Load Documents

In [7]:
## Dir PDF Loader
# loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
# loader = PyPDFLoader("./data/MedicalDocuments/orf-path_health-n1.pdf")
loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]

Number of chunks =  7
Interest rates are trimmed to 7.5 by the South African central bank,  but the lack of warning hits the rand and surprises markets.

The cost of buying both new and second hand cars fell sharply over the past five years, a new survey has found.

South Korea's central bank cuts interest rates by a quarter percentage point to 3.5 in a bid to drive growth in the economy.

An auction of shares in Google, the web search engine which could be floated for as much as \$36bn, takes place on Friday.

Hewlett-Packard shares fall after disappointing third-quarter profits, while the firm warns the final quarter will also fall short of expectations.

One of the oldest textile operators on the Indian Ocean island of Mauritius last week shut seven factories and cut 900 jobs.

Chad asks the IMF for a loan to pay for looking after more than 100,000 refugees from conflict-torn Darfur in western Sudan.

The company running the Japanese nuclear plant hit by a fatal accident is to close

## Create a dataframe of all the chunks

In [8]:
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(7, 3)


,text,source,chunk_id
0,"Reuters - Short-sellers, Wall Street's dwindli...",data/raw/input/ag_news/ag_news.txt,1cf992ac82a1413fa1495b9fdd30498e
1,Forbes.com - After earning a PH.D. in Sociolog...,data/raw/input/ag_news/ag_news.txt,228c82f0e68349448a062d6d5023dd9c
2,NEW YORK (Reuters) - The dollar tumbled broadl...,data/raw/input/ag_news/ag_news.txt,e6c2a9a61068450bb195b6438806655b
3,Interest rates are trimmed to 7.5 by the South...,data/raw/input/ag_news/ag_news.txt,1b6fd694933241bf9d8fd8ce696e6383
4,A group led by the UAE's Etisalat plans to spe...,data/raw/input/ag_news/ag_news.txt,5e4b8c2684e644cda5bee056564dfc0d


## Extract Concepts

If regenerate is set to True then the dataframes are regenerated and Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of edges

        df = dataframe of chunks


Else the dataframes are read from the output directory

In [9]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model='zephyr:latest')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

100%|█████████████████████████████████████████████| 7/7 [00:38<00:00,  5.50s/it]

Possibly due to JSON Decode Error, 4 chunks have been skipped
(28, 5)


,node_1,node_2,edge,chunk_id,count
0,forbes.com,ipo,The article mentions Google's highly anticipat...,228c82f0e68349448a062d6d5023dd9c,4
1,new york,stock market,Both New York (mentioned in the first article)...,228c82f0e68349448a062d6d5023dd9c,4
2,crude prices,oversupply,The third article mentions that OPEC can do no...,228c82f0e68349448a062d6d5023dd9c,4
3,non-opec oil exporters,crude prices,The fifth article suggests that Non-OPEC oil e...,228c82f0e68349448a062d6d5023dd9c,4
4,purnomo yusgiantoro,opec president,The third article mentions Purnomo Yusgiantoro...,228c82f0e68349448a062d6d5023dd9c,4


## Calculating contextual proximity

In [10]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
587,us economy,sidewalks,"40765f9cd43044a2ba9f7c348753d6e8,40765f9cd4304...",2,contextual proximity
588,us economy,state grant money,"40765f9cd43044a2ba9f7c348753d6e8,40765f9cd4304...",2,contextual proximity
589,us economy,temper,"40765f9cd43044a2ba9f7c348753d6e8,40765f9cd4304...",2,contextual proximity
590,us economy,turned the corner,"40765f9cd43044a2ba9f7c348753d6e8,40765f9cd4304...",2,contextual proximity
591,us economy,u-turn,"40765f9cd43044a2ba9f7c348753d6e8,40765f9cd4304...",2,contextual proximity


### Merge both the dataframes

In [11]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,annual base salary,crude prices,"228c82f0e68349448a062d6d5023dd9c,228c82f0e6834...",contextual proximity,2
1,annual base salary,danny bazil riley,"228c82f0e68349448a062d6d5023dd9c,228c82f0e6834...",The first article mentions Danny Bazil Riley's...,6
2,annual base salary,purnomo yusgiantoro,"228c82f0e68349448a062d6d5023dd9c,228c82f0e6834...",contextual proximity,2
3,annual base salary,stock market,"228c82f0e68349448a062d6d5023dd9c,228c82f0e6834...",contextual proximity,2
4,bill gates,central square in lynn,"40765f9cd43044a2ba9f7c348753d6e8,40765f9cd4304...",contextual proximity,14
...,...,...,...,...,...
208,us economy,sidewalks,"40765f9cd43044a2ba9f7c348753d6e8,40765f9cd4304...",contextual proximity,2
209,us economy,state grant money,"40765f9cd43044a2ba9f7c348753d6e8,40765f9cd4304...",contextual proximity,2
210,us economy,temper,"40765f9cd43044a2ba9f7c348753d6e8,40765f9cd4304...",contextual proximity,2
211,us economy,turned the corner,"40765f9cd43044a2ba9f7c348753d6e8,40765f9cd4304...",President Bush has been saying that the US eco...,6


## Calculate the NetworkX Graph

In [12]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(40,)

In [13]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

### Calculate communities for coloring the nodes

In [14]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  4
[['annual base salary', 'crude prices', 'danny bazil riley', 'financial planner', 'forbes.com', 'ipo', "iran's opec governor", 'new york', 'non-opec oil exporters', 'opec president', 'oversupply', 'purnomo yusgiantoro', 'short-sellers', 'stock market'], ['bill gates', 'central square in lynn', 'curbs', 'fences', 'former employees', 'gateway artisan block', 'government', 'landscaping', 'lights', 'lynn', 'marlborough-based technology company', 'oil', 'praise', 'road improvements', 'russia', 'sidewalks', 'state grant money', 'stein', 'temper', 'turned the corner', 'u-turn', 'us economy'], ['dell', 'kevin b. rollins'], ['gary winnick', 'global crossing']]


### Create a dataframe for community colors

In [15]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows.append({"node": node, "color": color, "group": group})
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,annual base salary,#57d3db,1
1,crude prices,#57d3db,1
2,danny bazil riley,#57d3db,1
3,financial planner,#57d3db,1
4,forbes.com,#57d3db,1
5,ipo,#57d3db,1
6,iran's opec governor,#57d3db,1
7,new york,#57d3db,1
8,non-opec oil exporters,#57d3db,1
9,opec president,#57d3db,1


### Add colors to the graph

In [16]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [17]:
from pyvis.network import Network

graph_output_directory = "./reports/graphs/graph.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory)

                                       
[Select a Node by ID                 ]
Reset Selection
